# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [ ]:
# class EnhancedLSTM(nn.Module):
#     def __init__(self, input_size=5, hidden_size=64, num_layers=2, dropout=0.2):
#         super().__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#
#         # 双向LSTM层
#         self.lstm = nn.LSTM(
#             input_size=input_size,
#             hidden_size=hidden_size,
#             num_layers=num_layers,
#             batch_first=True,
#             bidirectional=True,
#             dropout=dropout if num_layers > 1 else 0
#         )
#
#         # 注意力机制
#         self.attention = nn.Sequential(
#             nn.Linear(hidden_size*2, hidden_size),
#             nn.Tanh(),
#             nn.Linear(hidden_size, 1)
#         )
#
#         # 输出层
#         self.fc = nn.Sequential(
#             nn.Linear(hidden_size*2, hidden_size),
#             nn.Dropout(dropout),
#             nn.Linear(hidden_size, 1)
#         )
#
#     def forward(self, x):
#         # 初始化隐藏状态
#         h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)
#         c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)
#
#         # LSTM输出
#         lstm_out, _ = self.lstm(x, (h0, c0))
#
#         # 注意力权重
#         attn_weights = torch.softmax(self.attention(lstm_out), dim=1)
#         context = torch.sum(attn_weights * lstm_out, dim=1)
#
#         # 输出层
#         out = self.fc(context)
#         return out
#
# def enhanced_predict(df, target_col='收盘'):
#     """综合多特征、参数搜索和高级LSTM结构的增强预测函数"""
#     # 准备多特征数据
#     feature_cols = ['开盘', '最高', '最低', '收盘', '成交量']
#     target_idx = feature_cols.index(target_col)
#
#     # 数据标准化
#     scaler = MinMaxScaler()
#     scaled_data = scaler.fit_transform(df[feature_cols])
#
#     # 参数网格配置
#     param_grid = {
#         'hidden_size': [64, 128],
#         'num_layers': [2, 3],
#         'dropout': [0.2, 0.3],
#         'look_back': [30, 60],
#         'learning_rate': [0.001, 0.0005],
#         'epochs': [150],
#         'patience': [15]
#     }
#
#     best_rmse = float('inf')
#     best_params = {}
#     best_model_state = None
#     best_scaler = None
#     best_look_back = 0
#     best_predictions = None
#     best_actuals = None
#
#     # 参数搜索
#     for params in ParameterGrid(param_grid):
#         look_back = params['look_back']
#
#         # 创建序列数据集
#         X, y = [], []
#         for i in range(len(scaled_data) - look_back):
#             X.append(scaled_data[i:i+look_back])
#             y.append(scaled_data[i+look_back, target_idx])
#
#         if len(X) < 10:  # 跳过数据量不足的参数组合
#             continue
#
#         X = torch.FloatTensor(X)
#         y = torch.FloatTensor(y).view(-1, 1)
#
#         # 模型初始化
#         model = EnhancedLSTM(
#             input_size=len(feature_cols),
#             hidden_size=params['hidden_size'],
#             num_layers=params['num_layers'],
#             dropout=params['dropout']
#         )
#         optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
#         criterion = nn.MSELoss()
#
#         # 训练循环
#         best_train_loss = float('inf')
#         stop_counter = 0
#
#         for epoch in range(params['epochs']):
#             model.train()
#             optimizer.zero_grad()
#             outputs = model(X)
#             loss = criterion(outputs, y)
#             loss.backward()
#             torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#             optimizer.step()
#
#             # 早停机制
#             if loss.item() < best_train_loss:
#                 best_train_loss = loss.item()
#                 stop_counter = 0
#             else:
#                 stop_counter += 1
#                 if stop_counter >= params['patience']:
#                     break
#
#         # 评估模型
#         model.eval()
#         with torch.no_grad():
#             preds = model(X)
#             n_samples = len(preds)
#
#             # 反标准化处理
#             dummy_preds = np.zeros((n_samples, len(feature_cols)))
#             dummy_preds[:, target_idx] = preds.numpy().flatten()
#             preds_denorm = scaler.inverse_transform(dummy_preds)[:, target_idx]
#
#             dummy_y = np.zeros((n_samples, len(feature_cols)))
#             dummy_y[:, target_idx] = y.numpy().flatten()
#             y_denorm = scaler.inverse_transform(dummy_y)[:, target_idx]
#
#             rmse = np.sqrt(mean_squared_error(y_denorm, preds_denorm))
#
#             if rmse < best_rmse:
#                 best_rmse = rmse
#                 best_params = params
#                 best_model_state = model.state_dict()
#                 best_scaler = scaler
#                 best_look_back = look_back
#                 best_predictions = preds_denorm
#                 best_actuals = y_denorm
#
#     # 最终预测
#     final_model = EnhancedLSTM(
#         input_size=len(feature_cols),
#         hidden_size=best_params['hidden_size'],
#         num_layers=best_params['num_layers'],
#         dropout=best_params['dropout']
#     )
#     final_model.load_state_dict(best_model_state)
#     final_model.eval()
#
#     last_sequence = scaled_data[-best_look_back:]
#     input_tensor = torch.FloatTensor(last_sequence).view(1, best_look_back, -1)
#
#     with torch.no_grad():
#         next_pred_scaled = final_model(input_tensor).item()
#         dummy_next = np.zeros((1, len(feature_cols)))
#         dummy_next[0, target_idx] = next_pred_scaled
#         next_pred = best_scaler.inverse_transform(dummy_next)[0, target_idx]
#
#     # 输出结果
#     print(f"\n{target_col}价预测最佳参数: {best_params}")
#     print(f"训练集RMSE: {best_rmse:.2f}")
#     print(f"实际值={best_actuals[-1]:.2f} 预测值={best_predictions[-1]:.2f} (最新数据点)")
#     print(f"预测下一个交易日的{target_col}价可能为：{next_pred:.2f} 元")
#
#     return {
#         'best_params': best_params,
#         'rmse': best_rmse,
#         'last_actual': best_actuals[-1],
#         'last_pred': best_predictions[-1],
#         'next_pred': next_pred
#     }
#
# # 执行预测
# print("\n增强版多特征LSTM预测结果：")
# for col in ['开盘', '收盘', '最低', '最高']:
#     enhanced_predict(history_df, col)